In [6]:
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk

# Ruta al archivo CSV
file_path = 'cleaned_descriptions.csv'

# Leer el archivo CSV y convertirlo en un DataFrame
df = pd.read_csv(file_path)

# Mostrar las primeras filas del DataFrame
#df = df.drop(columns=['initial_description', 'final_description'])
#df = df.dropna(subset=['cleaned_final_description'])


In [8]:
# Descargar los recursos necesarios de nltk
nltk.download('punkt')

# Función para tokenizar y eliminar palabras repetidas
def tokenize_and_remove_duplicates(text):
    # Verificar si el texto es NaN o vacío
    if pd.isna(text) or text.strip() == "":
        return text  # Devolver el mismo valor si es NaN o vacío
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Eliminar palabras duplicadas manteniendo el orden
    unique_tokens = list(dict.fromkeys(tokens))
    return ' '.join(unique_tokens)

# Aplicar la función a cada fila de la columna 'cleaned_initial_description'
df['cleaned_initial_description'] = df['cleaned_initial_description'].apply(tokenize_and_remove_duplicates)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize

# Descargar los recursos necesarios de nltk
nltk.download('punkt')

# Combinar todas las descripciones en un solo texto
all_text = ' '.join(df['cleaned_initial_description'].dropna().tolist())

# Tokenizar el texto
tokens = word_tokenize(all_text)

# Calcular la frecuencia de cada palabra
word_freq = Counter(tokens)

# Convertir el resultado en un DataFrame
freq_df = pd.DataFrame(word_freq.items(), columns=['palabra', 'frecuencia'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler
import nltk

# Descargar los recursos necesarios de nltk
nltk.download('punkt')

# Ruta al archivo CSV de frecuencias
freq_file_path = 'freq.csv'

# Leer el archivo CSV de frecuencias y convertirlo en un DataFrame
freq_df = pd.read_csv(freq_file_path)

# Convertir el DataFrame de frecuencias en un diccionario para acceso rápido
frequency_dict = pd.Series(freq_df.frecuencia.values, index=freq_df.palabra).to_dict()

# Combinar todas las descripciones en un solo texto
all_text = ' '.join(df['cleaned_initial_description'].dropna().tolist())

# Tokenizar el texto
tokens = word_tokenize(all_text)

# Calcular la frecuencia de cada palabra en las descripciones
word_freq = Counter(tokens)

# Crear un DataFrame de las frecuencias calculadas
word_freq_df = pd.DataFrame(word_freq.items(), columns=['palabra', 'frecuencia'])

# Combinar las frecuencias de freq.csv y las calculadas
combined_freq_df = pd.merge(freq_df, word_freq_df, on='palabra', how='outer', suffixes=('_global', '_local')).fillna(0)

# Normalizar las frecuencias globales y locales
scaler = MinMaxScaler()
combined_freq_df[['frecuencia_global_normalizada', 'frecuencia_local_normalizada']] = scaler.fit_transform(
    combined_freq_df[['frecuencia_global', 'frecuencia_local']]
)

# Calcular la frecuencia balanceada como una media ponderada de las frecuencias normalizadas
combined_freq_df['frecuencia_balanceada'] = combined_freq_df[['frecuencia_global_normalizada', 'frecuencia_local_normalizada']].mean(axis=1)

# Convertir el DataFrame combinado en un diccionario para acceso rápido
combined_frequency_dict = pd.Series(combined_freq_df.frecuencia_balanceada.values, index=combined_freq_df.palabra).to_dict()

# Función para tokenizar, eliminar palabras repetidas y limitar a las más frecuentes con balance de frecuencias
def tokenize_and_limit_to_frequent(text, max_words=4):
    # Verificar si el texto es NaN o vacío
    if pd.isna(text) or text.strip() == "":
        return text  # Devolver el mismo valor si es NaN o vacío
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Eliminar palabras duplicadas manteniendo el orden
    unique_tokens = list(dict.fromkeys(tokens))
    # Ordenar las palabras por frecuencia balanceada, más frecuente primero
    sorted_tokens = sorted(unique_tokens, key=lambda word: combined_frequency_dict.get(word, 0), reverse=True)
    # Limitar el número de palabras a max_words
    limited_tokens = sorted_tokens[:max_words]
    return ' '.join(limited_tokens)

# Aplicar la función a cada fila de la columna 'cleaned_initial_description'
df['cleaned_initial_description'] = df['cleaned_initial_description'].apply(lambda x: tokenize_and_limit_to_frequent(x, max_words=4))

# Guardar el DataFrame actualizado
df.to_csv('archivo_actualizado.csv', index=False)

# Mostrar las primeras filas del DataFrame actualizado
print(df.head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: expected string or bytes-like object